# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name(s): HONGFEI YANG (783661)
###### Python version: 3.6

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the seven functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [1]:
import pandas as pd
import numpy as np
from itertools import permutations
import math
#lstOfCSV = ["breast-cancer.csv", "car.csv", "hypothyroid.csv", "mushroom.csv"]
lstOfCSV = ['train_top10.csv', 'dev_top10.csv', 'test_top10.csv']

In [9]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(csv_file_name):
    """
    preprocess the training dataset by droping all instances with '?' inside
    """
    df = pd.read_csv(csv_file_name, header=None)
    df = df.replace('?', np.NaN)
    df.dropna(inplace=True);
    
    return df


def preprocessTestData(csv_file_name):
    """
    preprocess test dataset by just reading it as a DataFrame
    """
    df = pd.read_csv(csv_file_name, header=None)
    return df

In [10]:
# This function should build a supervised NB model
def train_supervised(df):
    """
    return a trained NB supervised classifier.
    
    the classifier is a list of classes, in each class it has a list of dictionaries whose order
    corresponds to the order of attributes of the traing data , each dictionary has this attribute's
    values and their corresponding frequencies
    """
    
    def buildFreqDict(X, index):
        """
        return frequencies of the values of this attribute, which is at 'index' position.
        X is the data part of the training dataset
        """
        attrFreqDict = {}

        for i in range(len(X)):
            key = X[i][index]
            if key != "?":
                if key not in attrFreqDict:
                    attrFreqDict[key] = 0
                attrFreqDict[key] += 1

        return attrFreqDict

    def seperateByClass(df):
        """
        return a dictionary with its key being each class, value being a list of attributes which have thier own
        values dictionaries
        """
        values = df.values
        class_index = len(df.columns) - 1
        
        # seperate the dataset by data and groud truth
        X = values[:,:class_index]
        y = values[:, class_index]
        
        classDict = {}
        for i in range(len(values)):
            key = y[i]
            val = X[i]
            if key not in classDict:
                classDict[key] = []
            classDict[key].append(val)
        return classDict

    def buildClassiferDict(df):
        """
        return the final classsifier by combining all the data structures we have
        """
        
        numClass = len(df.columns) - 1
        
        classifierDict = {}
        
        classDict = seperateByClass(df)
        for y, X in classDict.items():
            classifierDict[y] = []
            for i in range(numClass):
                attrFreqDict = buildFreqDict(X, i)
                classifierDict[y].append(attrFreqDict)
            
            # we need to put the number of instances of this class for later use,
            # so we put it at the end of each class
            classifierDict[y].append(len(X))
        return classifierDict
    
    return buildClassiferDict(df)

#classifier = train_supervised(preprocess("breast-cancer.csv"))
#classifier

In [17]:
# This function should predict the class for a set of instances, based on a trained model 
def predict_supervised(classifier, testInstances, K=1):
    
    result = []
    
    for testInstanceIndex in range(len(testInstances)):
        
        testInstance = testInstances.iloc[testInstanceIndex, :]
        currScore = None
        outcome = None

        # the total number of instances we have is calculated by adding each class's number,
        # which is recorded at the end of the list of each class
        totalInstanceCount = sum([val[len(val) - 1] for val in classifier.values()])

        # calculate the probablity for each class to find the max score so we can determine what class it is
        for predictedClass, attributeFreqLst in classifier.items():

            # number of current class instances
            totalInstanceInClass = attributeFreqLst[len(attributeFreqLst) - 1]


            # beacause we are doing a lot of floating point multiplications that may affect accuracy,
            # so we make these operations more accurate by having a single division and many interger
            # multiplications
            posteriorProbNumerator = 1
            posteriorProbDenominator = 1

            # iterate through each attribute to calculate probabilities
            for i in range(len(testInstance)):
                #print(len(testInstance))
                # only care about fields with no missing data when predicting
                if testInstance[i] != '?':

                    # getting the total number of attribute values for this attribute, which is the |V| in the
                    # denominator of Laplace smoothing
                    lstOfAttributeExpr = set([item for sublst in [list(attriFreqLst[i].keys()) for attriFreqLst in classifier.values()] for item in sublst])
                    v = len(lstOfAttributeExpr)

                    # get the numerator & demoninator of each posterior probability and multiply them together respectively
                    # add one to each attribute value for Laplace smoothing
                    posteriorProbNumerator *= (attributeFreqLst[i].get(testInstance[i], 0) + K)
                    posteriorProbDenominator *= (totalInstanceInClass + v*K)

            # finally muliply by the prior probabilities and divde numerator and denominator
            probability = (totalInstanceInClass * posteriorProbNumerator) / (totalInstanceCount * posteriorProbDenominator)

            if currScore is None:
                # initiallise score here
                currScore = probability
                outcome = predictedClass
            elif probability > currScore:
                # remember the largest probability to determine the final classification
                currScore = probability
                outcome = predictedClass
        
        result.append(outcome)

    return result


#df = preprocess("breast-cancer.csv")

#testInstances = df.iloc[:1, :len(df.columns) - 1]
#predict_supervised(train_supervised(df), testInstances)

train_df = preprocess("train_top10.csv")
test_df = preprocess("dev_top10.csv")
test_df = test_df.iloc[:100,:len(test_df.columns)-1]
predict_supervised(train_supervised(train_df), test_df)


['24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '14-16',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '14-16',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26',
 '24-26']

In [12]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate_supervised(classifier, df):
    """
    evaluate supervised classifier by testing it again a set of data 'df'
    the accuracy is calculated by the number of correct prediction, divided by the
    total number of test instances
    """
    correctCount = 0
    totalCount = 0
   
            
    testInstances = df.iloc[:, :len(df.columns) - 1]
    result = predict_supervised(classifier, testInstances)
    answer = list(df.iloc[:, len(df.columns) - 1])

    for i in range(len(answer)):
        if result[i] == answer[i]:
            correctCount += 1
        totalCount += 1
        
    return correctCount / totalCount


def cross_validate_supervised(df, numPartition=10):
    """
    cross validate a supervised classifier
    """
    
    # shuffle the data first
    df = df.sample(frac=1).reset_index(drop=True)
    
    # determine the split side based on how many folds we want, default 10
    splitSize = math.ceil(len(df) / numPartition)
    
    
    startIndex = 0
    endIndex = splitSize-1
    accuracies = []
    for i in range(numPartition):
                
        # split dataset into test data and training data
        test_df = df.iloc[startIndex:endIndex, :]
        train_df = df.drop(df.index[startIndex:endIndex])
        
        # cross validation using training data to train the classifier
        # then use this to test against test data
        classifier = train_supervised(train_df)
        accuarcy = evaluate_supervised(classifier, test_df)
        
        accuracies.append(accuarcy)
        
        # adjust for next split
        startIndex = endIndex+1
        endIndex += splitSize-1
        
    
    # return the average of accuracies 
    return sum(accuracies)/numPartition


def get_metrics_supervised():
    """
    print the metrics of unsupervised NB for all four files
    """
    print("{:<20} {:<20} {:<20}".format(" ", "w/o CrossValidation", "w/ CrossValidation"))
    for oneFile in lstOfCSV:
        train_df = preprocessTestData(oneFile)
        test_df = train_df
        #test_df = preprocessTestData(oneFile)
        classifier = train_supervised(train_df)
        print("{:<20} {:<20.5f} {:<20.5f}".format(oneFile, evaluate_supervised(classifier, test_df), cross_validate_supervised(train_df)))
    return

get_metrics_supervised()



                     w/o CrossValidation  w/ CrossValidation  


KeyboardInterrupt: 

In [5]:
# This function should build an unsupervised NB model 
def train_unsupervised(df, NUM_ITER=10):
    """
    return a trained unsupervised classifier. number of iterations used to train is default to be 10
    """
    values = df.values
    
    numAttribute = len(df.columns) - 1 # number of attributes
    
    # divide dataset into data and groud truth
    X = values[:,:numAttribute]
    y = values[:,numAttribute]
    
    numClass = len(set(y)) # number of classes
    
    # calibrate a classifier using a matrix of probabilities
    def buildNewClassifier(X, labelProbMatrix, numClass=numClass, numAttribute=numAttribute):
        
        # this classifier is a list of list of dictionaries, each dictioary contain the total probabilities
        # a corresponding attribute value
        newClassifier = []
        
        # each attribute has a list of dictionary 
        for attributeIndex in range(numAttribute):

            attributeLabelProbDictList = []
            
            # for each label, get the total probabilities of each attribute value by looking through all
            # instances
            for labelIndex in range(numClass): 

                attributeDict = {}
                
                # look through all instances to update probabilities of each attribute values
                for instanceIndex in range(len(X)):

                    instance = X[instanceIndex]
                    attributeExpression = instance[attributeIndex]
                    
                    if attributeExpression != "?":
                        
                        if attributeExpression not in attributeDict:
                            attributeDict[attributeExpression] = 0

                        # add this correspoinding probability from the label probability matrix
                        attributeDict[attributeExpression] += labelProbMatrix[instanceIndex][labelIndex]

                attributeLabelProbDictList.append(attributeDict)

            newClassifier.append(attributeLabelProbDictList)
                
        return newClassifier

    # use the newly trained classier to get the label probabilty distribution matrix 
    def getNewLabelProbMatrix(X, classifier, labelProbMatrix, numClass=numClass, numAttribute=numAttribute):
        
        
        # this matrix has the same number of rows of the data, and its column number is the number of classes in this data
        newLabelProbMatrix = []

        for instanceIndex in range(len(X)):

            instance = X[instanceIndex]

            newLabelProbLst = []
            
            # process one column each time
            for labelIndex in range(numClass):
                
                # total probability of this label
                labelProbSum = sum([prob[labelIndex] for prob in labelProbMatrix])

                # prior probability
                newLabelProb = labelProbSum / len(X)

                # under this label, check each attribute
                for attributeIndex in range(numAttribute):

                    # attibute value of this instance
                    attributeExpression = instance[attributeIndex]

                    if attributeExpression != "?":
                        # get the posterior probability of this attribute of this instance:
                        #
                        # get the probabilities of this attribute of this label of this attribute value,
                        # divided by total probability of this label
                        attributePosteriorProb = classifier[attributeIndex][labelIndex].get(attributeExpression, 0) / labelProbSum

                        # muliply each conditional probability together
                        newLabelProb *= attributePosteriorProb

                newLabelProbLst.append(newLabelProb)
            
            # normalise
            newLabelProbLst = [i/sum(newLabelProbLst) for i in newLabelProbLst]

            newLabelProbMatrix.append(newLabelProbLst)

        return newLabelProbMatrix

    def convertToUnaryMatrix(labelProbMatrix):
        """
        convert n X numClass matrix to n X 1 matrix
        """
        
        unaryMatrix = []
        for row in labelProbMatrix:
            currMax = row[0]
            currMaxIndex = 0
            for i in range(1, len(row)):
                item = row[i]
                if item > currMax:
                    currMax = item
                    currMaxIndex = i
            # use the max probality's index as class name
            unaryMatrix.append(currMaxIndex)

        return unaryMatrix

    
    def buildFinalClassifier(X, unaryMatrix):
        """
        conbine our data with the newly formed unary matrix and
        train this data to get our final classifier that can give
        us a cluster number for a particular instance
        """
        _X = pd.DataFrame(X)
        _y = pd.DataFrame(unaryMatrix)

        newDF = pd.concat([_X, _y], axis=1, ignore_index=True)

        # train this as the same as supervised one
        classifier = train_supervised(newDF)

        return classifier
    
    
    # Initiallise the label probability matrix, 
    # We use a non-uniform distribution (Dirichlet Distribution) here. Total probability for each
    # row is 1
    labelProbMatrix = np.random.dirichlet(np.ones(numClass), size=len(df))
    
    # Train this data for serveral iterations
    i = NUM_ITER
    while (i>0):
        classifier = buildNewClassifier(X, labelProbMatrix)
        labelProbMatrix = getNewLabelProbMatrix(X, classifier, labelProbMatrix)
        i -= 1
    
    # and get our final classifier
    unaryMatrix = convertToUnaryMatrix(labelProbMatrix)
    finalClassifier = buildFinalClassifier(X, unaryMatrix)
    
    return finalClassifier

#df = preprocessTestData("breast-cancer.csv")
#classifier = train_unsupervised(df)
#classifier

In [6]:

def getMostAccurateClassification(classifier, training_df):
    """
    do swapping to get the most accurate classification for the data
    """    
    values = training_df.values
    
    numAttribute = len(training_df.columns) - 1

    X = values[:,:numAttribute]
    y = values[:,numAttribute]
    
    numClass = len(set(y))

    # set of classes
    classification = list(set(y))

    # do this for permuation because two or more clusters can be mapped to one class
    classification *= numClass
    
    # list of different classification
    possibleClassifications = list(permutations(classification, numClass))
    
    # get the unlabelled predictions
    unlabeledPrediction = predict_supervised(classifier, training_df.iloc[:, :numAttribute])

    # the calculate the most accurate labelling
    highestAccuracy = 0
    highestAccuracyClassification = None
    for oneClassification in possibleClassifications:
        
        # get one type of labelling
        predictionMapToClass = [oneClassification[unlabeledPrediction[i]] for i in range(len(unlabeledPrediction))]

        # get its accuracy
        accuracy = sum([1 if y[i] == predictionMapToClass[i] else 0 for i in range(len(unlabeledPrediction))]) / len(unlabeledPrediction)

        # keep the highest one
        if accuracy > highestAccuracy:
            highestAccuracy = accuracy
            highestAccuracyClassification = oneClassification

    assert(highestAccuracyClassification != None)
    
    return highestAccuracyClassification


# This function should predict the class distribution for a set of instances, based on a trained model
def predict_unsupervised(classifier, testInstance, classification):
    
    # get the unlabel result first
    unlabeledResult = predict_supervised(classifier, testInstance)
    
    # then use the most accurate classification we derived to label this result
    labeledResult = [classification[r] for r in unlabeledResult ]

    #return labeledResult
    return unlabeledResult, labeledResult

#df = preprocess("breast-cancer.csv")
#classifier = train_unsupervised(df)
#classification = getMostAccurateClassification(classifier, df)
#predict_unsupervised(classifier, df.iloc[:30, :len(df.columns)-1], classification)



In [7]:
# This function should evaluate a set of predictions, in an unsupervised manner
def evaluate_unsupervised(classifier, training_df, test_df):
    
    # ensure the list is not empty
    assert(not test_df.empty)
    
    # get the most accurate classification for this dataset
    classification = getMostAccurateClassification(classifier, training_df)
    
    correctCount = 0
    totalCount = 0

    answer = list(test_df.iloc[:, len(test_df.columns)-1])
        
    result, labeledResult = predict_unsupervised(classifier, test_df.iloc[:, :len(test_df.columns)-1], classification)

    # check the accuracy based on this classification we got earlier
    for i in range(len(answer)):
        
        if labeledResult[i] == answer[i]:
            correctCount += 1
        totalCount += 1
    
    accuarcy = correctCount / totalCount
    
    
    # build a confusion matrix using the unlabled data
    confusionDict = {}

    for i in range(len(answer)):
        
        if answer[i] not in confusionDict:
            confusionDict[answer[i]] = {}
            
        if result[i] not in confusionDict[answer[i]]:
            confusionDict[answer[i]][result[i]] = 0
            
        confusionDict[answer[i]][result[i]] += 1

        
    return confusionDict, classification, accuarcy


def cross_validate_unsupervised(df, numPartition=10):
    """
    cross validate an unsupervised classifier
    """
    
    # shuffle the data first
    df = df.sample(frac=1).reset_index(drop=True)
    
    # determine the split side based on how many folds we want, default 10
    splitSize = math.ceil(len(df) / numPartition)
    
    
    startIndex = 0
    endIndex = splitSize-1
    accuracies = []
    for i in range(numPartition):
        
        # split dataset into test data and training data
        test_df = df.iloc[startIndex:endIndex, :]
        train_df = df.drop(df.index[startIndex:endIndex])
        
        # cross validation by using the training data to train the classifier
        # then use test data to check the classifier performance
        classifier = train_unsupervised(train_df)
        confusionDict, classification, accuarcy = evaluate_unsupervised(classifier, train_df, test_df)
        
        accuracies.append(accuarcy)
        
        # adjust for next split
        startIndex = endIndex+1
        endIndex += splitSize-1
        
    
    # return the average of accuracies 
    return sum(accuracies)/numPartition

#df = preprocess("breast-cancer.csv")
#cross_validate_unsupervised(df)

In [8]:
def print_metrics_unsupervised():
    """
    print confusion matrix with other stats for all four datasets
    """
    
    for oneFile in lstOfCSV:
        
        # get stats first
        training_df = preprocess(oneFile)
        test_df = training_df
        #test_df = preprocessTestData(oneFile)
        classifier = train_unsupervised(training_df)
        confusionDict, classification, accuarcy = evaluate_unsupervised(classifier, training_df, test_df)
        
        output = "{:>20}".format(oneFile)
        for i in range(len(classification)):
            output += "{:>7}".format(i)
        output += "{:>7}".format("Total")
        print(output)
        output = "-"*(20+7*(len(classification)+1))
        print(output)
        for (key, val) in confusionDict.items():
            total = 0
            output = ""
            for i in range(len(classification)):
                output += "{:>7}".format(val.get(i, 0))
                total += val.get(i, 0)
            output += "{:>7}".format(total)
            print("{:>20}{}".format(key, output))

        output = "{:>20}".format("Total")

        totalTotal = 0
        for i in range(len(classification)):
            total = 0
            for (k, v) in confusionDict.items():
                total += v.get(i, 0)
            output += "{:>7}".format(total)
            totalTotal += total
        output += "{:>7}".format(totalTotal)
        print(output)
        output = "-"*(20+7*(len(classification)+1))
        print(output)
        output = ""
        for i in range(len(classification)):
            output += "{} -> {}    ".format(i, classification[i])
        print("Most accurate classification is:")
        print(output)
        print("Accuarcy:", accuarcy)
        print()
        
        
print_metrics_unsupervised()


KeyboardInterrupt: 

Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.